In [2]:
import os
import csv
import io
import json
from uuid import uuid4
from genson import SchemaBuilder
from flatten_json import flatten, unflatten
from datamodel_code_generator import DataModelType, PythonVersion
from datamodel_code_generator.model import get_data_model_types
from datamodel_code_generator.parser.jsonschema import JsonSchemaParser



def json_to_json_schema(input_json):
    input_dict = input_json.data
    builder = SchemaBuilder()
    builder.add_object(input_dict)
    json_schema = builder.to_schema()
    return json_schema